In [ ]:
import sys
print(sys.version)

In [ ]:
import pandas as pd
print(pd.__version__)

In [ ]:
pd.options.display.max_columns = None
pd.options.display.float_format = '{:,.2f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
csl_df = pd.read_sas('/home/mbopf/MFMDatasets/CSL_StudyItems/CSLDatasets/csllinkedbypreg.sas7bdat', format = 'sas7bdat', encoding='iso-8859-1')
csl_df.head()

In [ ]:
csl_df.info()

In [ ]:
#import numpy as np
#np.where(csl_df['Marital'].notnull() & csl_df['Marital'])
s = csl_df.Vbase3.apply(lambda x: pd.isnull(x) or (pd.notnull(x) and abs(x - int(x)) < 1e-6))
print(s)

In [ ]:
s = csl_df['Vbase3'].apply(lambda x: pd.notnull(x) and abs(x - int(x)) > 1e-6)
s.any()

In [ ]:
csl_df['Marital'].head()

In [ ]:
#csl_df.applymap(lambda x: str(int(x)) if abs(x - int(x)) < 1e-6 else str(round(x,2)))
#csl_df['Marital'].apply(lambda x: str(int(x)) if abs(x - int(x)) < 1e-6 else str(round(x,2)))

In [ ]:
def convert_integer (df):
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':
            # This is the really inefficient part
            for val in df[col]:
                if pd.notnull(val) and abs(val - int(val)) > 1e-6:
                    intcol_flag = False
                    break;
            if intcol_flag:
                print(f"{col} is integer")
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
            else:
                print(f"{col} is float")
        else:
            print(f"{col} is {df[col].dtype}")
    return df

In [ ]:
csl_df.agg(['count', 'size', 'nunique'])

In [ ]:
csl_df.describe()

In [ ]:
csl_df.agg(['min','max'])

In [ ]:
csl_df['Vbase3'][csl_df['Vbase3'].notnull()]

In [ ]:
csl_df.dtypes

In [3]:
def convert_integer (df):
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':
            for val in df[col]:
                if pd.notnull(val) and abs(val - int(val)) > 1e-6:
                    intcol_flag = False
                    break;
            if intcol_flag:
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
        print(f"{col} is {df[col].dtype}")
    return df

In [ ]:
df = convert_integer(csl_df)

In [ ]:
csl_df.columns

In [ ]:
csl_df.dtypes

In [ ]:
float(3.0).is_integer()

In [ ]:
csl_df['Marital'].value_counts(dropna=False)

In [ ]:
def int_columns (df, col_list):
    for col in col_list:
        df[col] = df[col].astype('Int16')
    return df
#int_columns(csl_df, ['Education','Insurance'])        

In [ ]:
csl_df.isnull().sum()

In [ ]:
csl_df.dtypes

In [ ]:
csl_df

In [ ]:
csl_df.index

In [ ]:
csl_df.columns

In [ ]:
csl_df.dtypes

In [ ]:
csl_df.shape

In [ ]:
csl_df.info()

In [ ]:
csl_df.describe(include='all')

In [ ]:
sites = csl_df.Sitenum
sites.value_counts().plot(kind='bar')

In [ ]:
csl_df['Sitenum'].head()

In [ ]:
csl_df.to_csv('/home/mbopf/MFMDatasets/CSL_StudyItems/mbopf/csl_all.csv')

In [ ]:
cols = csl_df.columns
print(cols)
[x for x in cols]

In [ ]:
csl_df['Sitenum']

In [1]:
# Clear some memory
import gc
del csl_df
gc.collect()
#df_formats = pd.read_sas('/home/mbopf/MFMDatasets/CSL_StudyItems/CSLDatasets/cslformats_deidentified.sas7bdat', format = 'sas7bdat', encoding='iso-8859-1')

NameError: name 'csl_df' is not defined

In [1]:
import numpy as np
import pandas as pd
np.random.seed(10)
df = pd.DataFrame(np.random.choice([1, 2, 3.3, 5000, 111111, np.NaN], (3,9)), 
                  columns=[f'col{i}' for i in range(9)])
df

,col0,col1,col2,col3,col4,col5,col6,col7,col8
0,2.0,NaN,111111.0,1.0,2.0,5000.0,111111.0,2.0,NaN
1,1.0,NaN,2.0,3.3,1.0,2.0,1.0,3.3,1.0
2,111111.0,5000.0,1.0,111111.0,5000.0,1.0,5000.0,3.3,2.0


In [4]:
convert_integer(df)

col0 is Int32
col1 is Int16
col2 is Int32
col3 is float64
col4 is Int16
col5 is Int16
col6 is Int32
col7 is float64
col8 is Int8


,col0,col1,col2,col3,col4,col5,col6,col7,col8
0,2,NaN,111111,1.0,2,5000,111111,2.0,NaN
1,1,NaN,2,3.3,1,2,1,3.3,1
2,111111,5000,1,111111.0,5000,1,5000,3.3,2


In [13]:
df.max(axis=1)

0    1231231.0
1    1231231.0
2    1231231.0
dtype: float64

In [14]:
s = pd.cut(df.max(), bins=[0, 127, 32767, 2147483647], labels=['Int8', 'Int16', 'Int32'])
print(s)
s = s.where(df.dtypes=='float')  # So we only cast the previously float cols 
print(s)

#for idx, gp in s.groupby(s):
#    df.loc[:, gp.index] = df.loc[:, gp.index].astype(idx)

col0     Int32
col1     Int16
col2     Int32
col3     Int16
col4      Int8
col5      Int8
col6     Int32
col7     Int32
col8     Int16
col9     Int32
col10    Int32
col11    Int16
col12    Int32
col13    Int32
col14     Int8
col15    Int32
col16     Int8
col17    Int32
col18    Int32
col19    Int32
dtype: category
Categories (3, object): [Int8 < Int16 < Int32]
col0     Int32
col1     Int16
col2     Int32
col3     Int16
col4      Int8
col5      Int8
col6     Int32
col7     Int32
col8     Int16
col9     Int32
col10    Int32
col11    Int16
col12    Int32
col13    Int32
col14     Int8
col15    Int32
col16     Int8
col17    Int32
col18    Int32
col19    Int32
dtype: category
Categories (3, object): [Int8 < Int16 < Int32]
